In [1]:
%pip install -q kagglehub
import kagglehub
import tensorflow as tf
import cv2
import numpy as np
import os

# Download latest version of the MoveNet Thunder model
model_dir = kagglehub.model_download("google/movenet/tensorFlow2/singlepose-thunder")
# model_dir will be something like:
#   "C:/Users/YourUserName/.cache/kagglehub/models/google/movenet/tensorFlow2/singlepose-thunder"

# Sometimes MoveNet model files are stored under a subdirectory named "4"
# Check if "4" exists in model_dir; if so, point model_path to that subdirectory:
model_path = os.path.join(model_dir, "4")
if not os.path.exists(model_path):
    # If there's no '4' subfolder, we can just use model_dir directly:
    model_path = model_dir

# Load the saved model
model = tf.saved_model.load(model_path)
movenet = model.signatures['serving_default']


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\assaf\Documents\code\neural\final\tf-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Note: you may need to restart the kernel to use updated packages.


In [14]:
import numpy as np
import cv2
import tensorflow as tf

def preprocess_frame(frame):
    # Resize and normalize image
    img = tf.image.resize_with_pad(tf.expand_dims(frame, axis=0), 256, 256)
    img = tf.cast(img, dtype=tf.int32)
    return img

def draw_keypoints(frame, keypoints, confidence_threshold=0.3):
    h, w, _ = frame.shape
    for kp in keypoints[0, 0, :, :]:
        y, x, c = kp
        if c > confidence_threshold:
            cx, cy = int(x * w), int(y * h)
            cv2.circle(frame, (cx, cy), 4, (0, 255, 0), -1)

# Initialize video capture
cap = cv2.VideoCapture("./videos/good_form/bench press_38.mp4")  # Or your specific video file
data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to RGB and preprocess
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    input_image = preprocess_frame(rgb)

    # Run pose estimation
    outputs = movenet(input_image)
    keypoints = outputs['output_0'].numpy()

    flat_input = keypoints[0, 0, :, :2].flatten()  # shape (34,)
    data.append(flat_input)

    # Draw keypoints for visualization
    draw_keypoints(frame, keypoints)

    # Show frame
    cv2.imshow('MoveNet Thunder', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

# Save the full sequence to a single .npy file
data = np.array(data)  # shape (num_frames, 34)
np.save('./file1.npy', data)